In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product
import multiprocessing as mp
import math

In [2]:
start = time.time()
# import nodes
road_node = gpd.read_file(r'D:\Dumps\Philadelphia 2km buffer_all\nodes.shp')
road_node = road_node.set_index('osmid')
road_nodes = road_node.to_crs(4326)
road_nodes['geometry_m'] = gpd.GeoSeries(road_nodes['geometry'], crs = 4326).to_crs(3043)
road_nodes

,y,x,street_cou,lon,lat,highway,ref,geometry,geometry_m
osmid,,,,,,,,,
103237949,4.431781e+06,503033.785157,3,-74.964440,40.036252,None,None,POINT (-74.96444 40.03625),POINT (-5689489.546 8448457.709)
103353127,4.431724e+06,502937.820011,3,-74.965565,40.035733,None,None,POINT (-74.96557 40.03573),POINT (-5689617.107 8448567.833)
103353090,4.431839e+06,503124.706820,3,-74.963374,40.036770,None,None,POINT (-74.96337 40.03677),POINT (-5689364.441 8448354.775)
103237976,4.431638e+06,503122.315306,4,-74.963403,40.034959,None,None,POINT (-74.96340 40.03496),POINT (-5689653.040 8448263.511)
103238007,4.431510e+06,503203.675365,4,-74.962450,40.033810,None,None,POINT (-74.96245 40.03381),POINT (-5689797.056 8448087.089)
...,...,...,...,...,...,...,...,...,...
9703813717,4.416438e+06,483689.901846,3,-75.190791,39.897861,None,None,POINT (-75.19079 39.89786),POINT (-5720575.973 8468970.364)
9703813743,4.416385e+06,483760.410069,3,-75.189965,39.897385,None,None,POINT (-75.18996 39.89738),POINT (-5720619.056 8468844.195)
9703813748,4.416433e+06,484049.808886,3,-75.186581,39.897820,None,None,POINT (-75.18658 39.89782),POINT (-5720414.758 8468450.854)


In [3]:
# import edges
road_edge = gpd.read_file(r'D:\Dumps\Philadelphia 2km buffer_all\edges.shp')
road_edge = road_edge.set_index(['u','v','key'])
road_edges = road_edge.to_crs(4326)
road_edges['geometry_m'] = gpd.GeoSeries(road_edges['geometry'], crs = 4326).to_crs(3043)
road_edges

#Get both ways
RE = road_edges.reset_index()
RE2 = RE[['osmid','from','to','name','length','geometry']]
RE2.columns = ['osmid','to','from','name','length','geometry']
RE2 = RE2[['osmid','from','to','name','length','geometry']]
road_connections = pd.concat([RE, RE2]).to_crs(4326)
road_connections['key'] = road_connections['from'].astype(str) + '-' + road_connections['to'].astype(str)

In [4]:
road_edges[road_edges['from'] == 8627877668]

,,,osmid,name,highway,oneway,length,from,to,ref,lanes,bridge,service,access,width,tunnel,junction,maxspeed,area,geometry,geometry_m
u,v,key,,,,,,,,,,,,,,,,,,,
8050448583,8627877668,0,"[108593573, 688421782]",Port Royal Avenue,residential,0,309.651,8627877668,8050448583,None,None,None,None,None,None,None,None,None,None,"LINESTRING (-75.25214 40.05009, -75.25227 40.0...","LINESTRING (-5698738.706 8484331.709, -5698758..."
4771110405,8627877668,0,108593573,Port Royal Avenue,residential,0,459.237,8627877668,4771110405,None,None,None,None,None,None,None,None,None,None,"LINESTRING (-75.25214 40.05009, -75.25197 40.0...","LINESTRING (-5698738.706 8484331.709, -5698712..."


In [5]:
# Get the graph object
graph = ox.graph_from_place('Philadelphia, United States', network_type="all", buffer_dist = 2000)
graph = ox.speed.add_edge_speeds(graph)
graph = ox.speed.add_edge_travel_times(graph)

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [6]:
# Import greenspace and population grids
Penn_bound = gpd.read_file(r'C:\Users\bartb\Downloads\Pennsylvania\boundaries.shp')
Philly_greenspace = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\Public_greenspace.gpkg')
Philly_popgrid = gpd.read_file(r'C:\Users\bartb\Downloads\OneDrive_1_03-05-2022\CPoPGrid.gpkg')
Philly_bound = Penn_bound[Penn_bound['municipal1'] == 'PHILADELPHIA']

In [7]:
# Set crs to WGS84
Philly_bound = Philly_bound.to_crs(4326)
Philly_greenspace = Philly_greenspace.to_crs(4326)
Philly_popgrid = Philly_popgrid.to_crs(4326)

In [8]:
# Set buffer. Crs 3043 measures the world in metres, you can set a buffer in metres and set the data again in WGS84.
green_buffer = gpd.GeoDataFrame(geometry = Philly_greenspace.to_crs(3043).buffer(25).to_crs(4326))

# This function group components in itself that overlap (with the buffer set of 25 metres)
# https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
W = libpysal.weights.fuzzy_contiguity(green_buffer)
Philly_greenspace['components'] = W.component_labels
Philly_parks = Philly_greenspace.dissolve('components')

# Exclude parks below 0.04 ha.
Philly_parks = Philly_parks[Philly_parks.to_crs(3043).area > 400]

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\libpysal\weights\weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 747 disconnected components.
 There are 585 islands with ids: 11, 15, 26, 28, 45, 52, 53, 61, 65, 68, 71, 89, 90, 91, 95, 96, 97, 98, 103, 104, 105, 106, 107, 108, 110, 117, 118, 127, 133, 134, 138, 139, 148, 149, 153, 154, 155, 156, 157, 159, 163, 168, 171, 173, 174, 177, 181, 182, 185, 186, 189, 190, 191, 192, 193, 195, 197, 198, 199, 200, 201, 202, 203, 217, 221, 228, 232, 235, 239, 240, 241, 242, 245, 249, 255, 256, 261, 262, 264, 266, 267, 272, 273, 275, 276, 280, 285, 286, 288, 290, 292, 301, 303, 304, 307, 310, 313, 315, 316, 318, 320, 321, 322, 326, 327, 328, 329, 330, 331, 332, 335, 336, 337, 341, 342, 343, 344, 348, 349, 351, 355, 356, 357, 358, 359, 362, 363, 370, 371, 385, 437, 445, 449, 450, 451, 460, 461, 462, 463, 466, 467, 473, 483, 484, 485, 497, 498, 499, 503, 504, 547, 562, 564, 566, 569, 578, 579, 581, 582, 583, 5

In [9]:
# Make sure only complete overlays progress and take their centroids
popgrid = Philly_popgrid.overlay(Philly_bound)
popgrid['centroid'] = popgrid.to_crs(4326).centroid
popgrid['centroid_m'] = popgrid['centroid'].to_crs(3043)
popgrid['area'] = popgrid.area / popgrid.area.max()
popgrid = popgrid[(popgrid['area'] >= 0.99) & popgrid['PoP2015_Number'] > 0]
popgrid['grid_lon'] = popgrid['centroid_m'].x
popgrid['grid_lat'] = popgrid['centroid_m'].y
popgrid = popgrid.reset_index()
popgrid

C:\Users\bartb\AppData\Local\Temp\ipykernel_7668\1366358376.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  popgrid['centroid'] = popgrid.to_crs(4326).centroid
C:\Users\bartb\AppData\Local\Temp\ipykernel_7668\1366358376.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  popgrid['area'] = popgrid.area / popgrid.area.max()


,index,grid_id,PoP2015_Number,class_of_m,county,fed_aid_ur,fed_id_num,fips_area,fips_count,fips_mun_c,...,gpid,mslink,municipal,municipal1,geometry,centroid,centroid_m,area,grid_lon,grid_lat
0,6,259,113,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01452 40.13521, -75.01452 40.133...",POINT (-75.01564 40.13435),POINT (-5675978.406 8459812.552),0.996208,-5.675978e+06,8.459813e+06
1,7,260,135,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01227 40.13521, -75.01227 40.133...",POINT (-75.01339 40.13435),POINT (-5675888.831 8459539.227),0.996208,-5.675889e+06,8.459539e+06
2,13,400,137,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01676 40.13349, -75.01676 40.131...",POINT (-75.01789 40.13262),POINT (-5676341.311 8459996.328),0.996234,-5.676341e+06,8.459996e+06
3,16,403,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.01003 40.13349, -75.01003 40.131...",POINT (-75.01115 40.13262),POINT (-5676072.561 8459176.309),0.996234,-5.676073e+06,8.459176e+06
4,17,404,109,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.00778 40.13349, -75.00778 40.131...",POINT (-75.00890 40.13262),POINT (-5675982.949 8458902.986),0.996234,-5.675983e+06,8.458903e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3886,10223,20883,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25706 39.88308, -75.25706 39.881...",POINT (-75.25819 39.88221),POINT (-5725754.488 8476450.010),0.999924,-5.725754e+06,8.476450e+06
3887,10225,20885,39,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25257 39.88308, -75.25257 39.881...",POINT (-75.25369 39.88221),POINT (-5725576.204 8475897.317),0.999924,-5.725576e+06,8.475897e+06
3888,10226,20886,5,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25033 39.88308, -75.25033 39.881...",POINT (-75.25145 39.88221),POINT (-5725487.039 8475620.982),0.999924,-5.725487e+06,8.475621e+06
3889,10275,21024,3,CITY,67,4,23-6003047,00004,101,60000,...,39973.0,2573.0,301,PHILADELPHIA,"POLYGON ((-75.25931 39.88135, -75.25931 39.879...",POINT (-75.26043 39.88048),POINT (-5726119.978 8476637.252),0.999949,-5.726120e+06,8.476637e+06


In [10]:
# Take a buffer for the parks to ensure equal measurements for edge population grids
boundaries_buffer = gpd.GeoDataFrame(geometry = Philly_bound.to_crs(3043).buffer(1000).to_crs(4326))
parks_within_range = Philly_parks.overlay(boundaries_buffer)
parks_within_range['centroid'] = parks_within_range.centroid
parks_within_range['centroid_m'] = parks_within_range['centroid'].to_crs(3043)
parks_within_range

C:\Users\bartb\AppData\Local\Temp\ipykernel_7668\1721483655.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parks_within_range['centroid'] = parks_within_range.centroid


,OBJECTID,FeatClass,Category,Own_Type,Own_Name,Loc_Own,Mang_Type,Mang_Name,Loc_Mang,Des_Tp,...,IUCNCtDt,Date_Est,Comments,EsmtHldr,EHoldTyp,SHAPE_Leng,SHAPE_Area,geometry,centroid,centroid_m
0,81288,Fee,Fee,LOC,CNTY,Montgomery County,LOC,CNTY,Montgomery County,LOTH,...,2020,1983,None,Natural Lands Trust,NGO,2167.370365,1.070843e+05,"MULTIPOLYGON (((-75.24324 40.05599, -75.24326 ...",POINT (-75.25165 40.05653),POINT (-5697693.589 8484600.730)
1,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,LOTH,...,2020,None,None,None,None,36128.050943,3.353065e+06,"POLYGON ((-75.24706 39.91693, -75.24708 39.916...",POINT (-75.24714 39.91942),POINT (-5719375.750 8477008.358)
2,82731,Fee,Fee,LOC,CNTY,Philadelphia County,LOC,CNTY,Philadelphia County,LOTH,...,2020,None,None,None,None,36128.050943,3.353065e+06,"MULTIPOLYGON (((-75.23911 39.94226, -75.23909 ...",POINT (-75.25640 39.96241),POINT (-5712881.953 8480355.991)
3,1260,Fee,Fee,FED,NPS,NPS,FED,NPS,NPS,HCA,...,2020,1978,Edgar Allan Poe National Historic Site,None,None,216.240707,2.114309e+03,"POLYGON ((-75.14993 39.96159, -75.15020 39.961...",POINT (-75.15001 39.96195),POINT (-5708732.995 8467282.252)
4,1348,Fee,Fee,FED,NPS,NPS,FED,NPS,NPS,HCA,...,2020,None,Independence National Historical Park,None,None,6162.483495,1.285207e+05,"MULTIPOLYGON (((-75.17209 40.03406, -75.17261 ...",POINT (-75.17237 40.03387),POINT (-5698171.846 8473736.251)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,242021,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,Upper Darby Township,LP,...,2020,None,None,None,None,217.559826,1.590417e+03,"POLYGON ((-75.25807 39.94941, -75.25801 39.949...",POINT (-75.25763 39.94953),POINT (-5714985.845 8479845.686)
470,242022,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,Upper Darby Township,LP,...,2020,None,None,None,None,315.117880,4.314566e+03,"POLYGON ((-75.27571 39.96864, -75.27582 39.968...",POINT (-75.27612 39.96913),POINT (-5712590.186 8483121.334)
471,242023,Fee,Fee,LOC,CITY,Upper Darby Township,LOC,CITY,Upper Darby Township,LP,...,2020,None,None,None,None,458.291779,1.188739e+04,"POLYGON ((-75.25671 39.94612, -75.25686 39.947...",POINT (-75.25609 39.94665),POINT (-5715384.150 8479507.849)
472,242035,Fee,Fee,LOC,CITY,UPPER DARBY TOWNSHIP,LOC,CITY,UPPER DARBY TOWNSHIP,LP,...,2020,None,None,None,None,552.520121,1.592591e+04,"POLYGON ((-75.27647 39.96684, -75.27650 39.966...",POINT (-75.27714 39.96644),POINT (-5713059.443 8483108.295)


In [11]:
# Buffer on park edges
parks_buffer = gpd.GeoDataFrame(geometry = parks_within_range.to_crs(3043).buffer(25).to_crs(4326))
parks_buffer.to_file(r'C:\Users\bartb\Downloads\parks_buffer.shp')

# Check intersecting road nodes.
park_to_road = gpd.sjoin(parks_buffer, road_nodes.reset_index(), how = 'inner')
park_to_road = park_to_road.reset_index()
park_to_road['park_lon'] = park_to_road['geometry_m'].x
park_to_road['park_lat'] = park_to_road['geometry_m'].y
park_to_road

C:\Users\bartb\miniconda3\envs\ssml\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


,index,geometry,index_right,osmid,y,x,street_cou,lon,lat,highway,ref,geometry_m,park_lon,park_lat
0,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",77946,8627877668,4.433348e+06,478492.941523,3,-75.252142,40.050095,None,None,POINT (-5698738.706 8484331.709),-5.698739e+06,8.484332e+06
1,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",72001,5005312359,4.433861e+06,478532.801723,4,-75.251692,40.054719,None,None,POINT (-5697984.550 8484513.413),-5.697985e+06,8.484513e+06
2,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",71988,4244000096,4.433874e+06,478650.618839,4,-75.250311,40.054840,None,None,POINT (-5697910.887 8484350.631),-5.697911e+06,8.484351e+06
3,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",71994,5005312355,4.433890e+06,478614.288080,3,-75.250737,40.054987,None,None,POINT (-5697904.199 8484410.375),-5.697904e+06,8.484410e+06
4,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",71991,6651751695,4.433918e+06,478652.021991,3,-75.250296,40.055237,None,None,POINT (-5697847.121 8484369.113),-5.697847e+06,8.484369e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15187,472,"POLYGON ((-75.27647 39.96699, -75.27645 39.966...",4498,3425873720,4.424118e+06,476299.210585,3,-75.277524,39.966878,None,None,POINT (-5713004.185 8483178.234),-5.713004e+06,8.483178e+06
15188,473,"POLYGON ((-75.27364 39.96309, -75.27341 39.963...",24108,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-5713613.274 8482370.035),-5.713613e+06,8.482370e+06
15189,473,"POLYGON ((-75.27364 39.96309, -75.27341 39.963...",125822,8932010102,4.423606e+06,476578.411994,1,-75.274236,39.962271,None,None,POINT (-5713609.358 8482538.363),-5.713609e+06,8.482538e+06
15190,473,"POLYGON ((-75.27364 39.96309, -75.27341 39.963...",58595,923789126,4.423638e+06,476591.389933,3,-75.274085,39.962560,None,None,POINT (-5713557.281 8482534.680),-5.713557e+06,8.482535e+06


In [12]:
parks_buffer

,geometry
0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049..."
1,"POLYGON ((-75.24696 39.91680, -75.24697 39.916..."
2,"POLYGON ((-75.24849 39.92171, -75.24843 39.921..."
3,"POLYGON ((-75.14974 39.96156, -75.14974 39.961..."
4,"POLYGON ((-75.17280 40.03368, -75.17280 40.033..."
...,...
469,"POLYGON ((-75.25806 39.94926, -75.25808 39.949..."
470,"POLYGON ((-75.27554 39.96857, -75.27555 39.968..."
471,"POLYGON ((-75.25669 39.94597, -75.25671 39.945..."
472,"POLYGON ((-75.27647 39.96699, -75.27645 39.966..."


In [13]:
# Check example (with QGIS)
park_to_road[park_to_road['index'] == 453]

,index,geometry,index_right,osmid,y,x,street_cou,lon,lat,highway,ref,geometry_m,park_lon,park_lat
15047,453,"POLYGON ((-75.08029 40.01353, -75.08031 40.013...",33027,110291816,4.429273e+06,493150.989575,4,-75.080253,40.013627,None,None,POINT (-5697726.578 8461422.644),-5.697727e+06,8.461423e+06
15048,453,"POLYGON ((-75.08029 40.01353, -75.08031 40.013...",16998,109890052,4.429296e+06,493169.700669,3,-75.080034,40.013838,None,None,POINT (-5697684.279 8461406.828),-5.697684e+06,8.461407e+06
15049,453,"POLYGON ((-75.08029 40.01353, -75.08031 40.013...",24828,110188472,4.429317e+06,493092.570321,3,-75.080938,40.014029,None,None,POINT (-5697690.013 8461527.279),-5.697690e+06,8.461527e+06


In [14]:
PtR = pd.merge(park_to_road, road_nodes['geometry'], left_on = 'osmid', right_index = True)
PtR = gpd.GeoDataFrame(PtR,geometry = 'geometry_y', crs = 4326)
PtR['geometry_m'] = PtR['geometry_y'].to_crs(3043)
PtR

,index,geometry_x,index_right,osmid,y,x,street_cou,lon,lat,highway,ref,geometry_m,park_lon,park_lat,geometry_y
0,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",77946,8627877668,4.433348e+06,478492.941523,3,-75.252142,40.050095,None,None,POINT (-5698738.706 8484331.709),-5.698739e+06,8.484332e+06,POINT (-75.25214 40.05009)
1,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",72001,5005312359,4.433861e+06,478532.801723,4,-75.251692,40.054719,None,None,POINT (-5697984.550 8484513.413),-5.697985e+06,8.484513e+06,POINT (-75.25169 40.05472)
2,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",71988,4244000096,4.433874e+06,478650.618839,4,-75.250311,40.054840,None,None,POINT (-5697910.887 8484350.631),-5.697911e+06,8.484351e+06,POINT (-75.25031 40.05484)
3,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",71994,5005312355,4.433890e+06,478614.288080,3,-75.250737,40.054987,None,None,POINT (-5697904.199 8484410.375),-5.697904e+06,8.484410e+06,POINT (-75.25074 40.05499)
4,0,"POLYGON ((-75.25690 40.04984, -75.25706 40.049...",71991,6651751695,4.433918e+06,478652.021991,3,-75.250296,40.055237,None,None,POINT (-5697847.121 8484369.113),-5.697847e+06,8.484369e+06,POINT (-75.25030 40.05524)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15187,472,"POLYGON ((-75.27647 39.96699, -75.27645 39.966...",4498,3425873720,4.424118e+06,476299.210585,3,-75.277524,39.966878,None,None,POINT (-5713004.185 8483178.234),-5.713004e+06,8.483178e+06,POINT (-75.27752 39.96688)
15188,473,"POLYGON ((-75.27364 39.96309, -75.27341 39.963...",24108,110006563,4.423569e+06,476683.632252,3,-75.273003,39.961942,None,None,POINT (-5713613.274 8482370.035),-5.713613e+06,8.482370e+06,POINT (-75.27300 39.96194)
15189,473,"POLYGON ((-75.27364 39.96309, -75.27341 39.963...",125822,8932010102,4.423606e+06,476578.411994,1,-75.274236,39.962271,None,None,POINT (-5713609.358 8482538.363),-5.713609e+06,8.482538e+06,POINT (-75.27424 39.96227)
15190,473,"POLYGON ((-75.27364 39.96309, -75.27341 39.963...",58595,923789126,4.423638e+06,476591.389933,3,-75.274085,39.962560,None,None,POINT (-5713557.281 8482534.680),-5.713557e+06,8.482535e+06,POINT (-75.27409 39.96256)


In [15]:
# Check all parks within 1000m radius
start_time = time.time()
len1 = len(popgrid)
len2 = len(park_to_road)
len3 = int(np.ceil(len(park_to_road)/1000))
output = pd.DataFrame()
len_mat = 0
# Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
for i in range(len3):
    # Check all grid-park combinations per 1000
    l1, l2 = range(0,len1), range(i*1000,(i+1)*1000)
    listed = pd.DataFrame(list(product(l1, l2)))
    
    # Merge grid and park information
    grid_merged = pd.merge(listed, 
                           popgrid[['grid_id','centroid','centroid_m','grid_lon','grid_lat']], left_on = 0, right_index = True)
    node_merged = pd.merge(grid_merged, 
                           PtR[['osmid','geometry_y','geometry_m','park_lon','park_lat']], left_on = 1, right_index = True)
    
    # Preset index for merging
    len4 = len(node_merged)
    node_merged['key'] = range(0,len(node_merged))
    node_merged = node_merged.set_index('key')
    node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]
    #node_merged = node_merged.reset_index()
    
    # Create lists for better computational performance
    glon = list(node_merged['grid_lon'])
    glat = list(node_merged['grid_lat'])
    plon = list(node_merged['park_lon'])
    plat = list(node_merged['park_lat'])
    
    # Get the euclidean distances
    mat = np.repeat(None,len(node_merged))
    for j in range(len(node_merged)):
        mat[j] = math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2)
    
    # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
    mat_df = pd.DataFrame(mat)
    #mat_df = mat_df.reset_index()
    mat_df = mat_df[mat_df <=1000].dropna()
    mat_df = pd.DataFrame(mat_df)
    mat_df.columns = ['euclidean']    
    mat_df = mat_df.join(node_merged)
    
    output = pd.concat([output, mat_df])

    print((i+1),'/',len3,'done',round((time.time() - start_time) / 60,2),' mns')
    print('of',np.where(i+1 == len3, len2 % 1000 * 1000, len1*1000) ,'within 1000m:',len(mat_df))
    
    # Checks the number of the parks within 1000m.
    len_mat = len_mat + len(mat_df)
    
# Renaming columns
print('total combinations within distance',len_mat)
output.columns = ['euclidean','Grid_No','Park_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid','grid_lon','grid_lat',
                  'Parkroad_osmid','Parkroad_coords_centroid','Parkroad_m_centroid','park_lon','park_lat']
output = output[['euclidean','Grid_No','Park_No','Grid_Id','Grid_coords_centroid','Grid_m_centroid',
                  'Parkroad_osmid','Parkroad_coords_centroid','Parkroad_m_centroid']]
output
    

1 / 16 done 0.16  mns
of 3891000 within 1000m: 12795
2 / 16 done 0.33  mns
of 3891000 within 1000m: 13043
3 / 16 done 0.5  mns
of 3891000 within 1000m: 18233
4 / 16 done 0.66  mns
of 3891000 within 1000m: 13603
5 / 16 done 0.83  mns
of 3891000 within 1000m: 11841
6 / 16 done 0.99  mns
of 3891000 within 1000m: 10986
7 / 16 done 1.16  mns
of 3891000 within 1000m: 15181
8 / 16 done 1.34  mns
of 3891000 within 1000m: 12884
9 / 16 done 1.51  mns
of 3891000 within 1000m: 15565
10 / 16 done 1.69  mns
of 3891000 within 1000m: 13160
11 / 16 done 1.85  mns
of 3891000 within 1000m: 17386
12 / 16 done 2.03  mns
of 3891000 within 1000m: 17190
13 / 16 done 2.19  mns
of 3891000 within 1000m: 18591
14 / 16 done 2.35  mns
of 3891000 within 1000m: 18389
15 / 16 done 2.52  mns
of 3891000 within 1000m: 17289
16 / 16 done 2.6  mns
of 192000 within 1000m: 1833
total combinations within distance 227969


,euclidean,Grid_No,Park_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Parkroad_osmid,Parkroad_coords_centroid,Parkroad_m_centroid
1151,710.210312,1151,0,6972,POINT (-75.24696 40.05156),POINT (-5698301.554 8483771.980),8627877668,POINT (-75.25214 40.05009),POINT (-5698738.706 8484331.709)
1152,986.285677,1152,0,6973,POINT (-75.24471 40.05156),POINT (-5698213.121 8483497.131),8627877668,POINT (-75.25214 40.05009),POINT (-5698738.706 8484331.709)
1299,627.878287,1299,0,7396,POINT (-75.25145 40.04638),POINT (-5699303.046 8484056.480),8627877668,POINT (-75.25214 40.05009),POINT (-5698738.706 8484331.709)
1300,727.424412,1300,0,7397,POINT (-75.24920 40.04638),POINT (-5699214.620 8483781.571),8627877668,POINT (-75.25214 40.05009),POINT (-5698738.706 8484331.709)
1301,911.496244,1301,0,7398,POINT (-75.24696 40.04638),POINT (-5699126.179 8483506.669),8627877668,POINT (-75.25214 40.05009),POINT (-5698738.706 8484331.709)
...,...,...,...,...,...,...,...,...,...
528005,913.539094,2720,15135,12225,POINT (-75.24920 39.98767),POINT (-5708569.997 8480770.261),3618217765,POINT (-75.24980 39.99311),POINT (-5707727.165 8481122.665)
531860,982.48177,2684,15136,12085,POINT (-75.24471 39.98940),POINT (-5708117.175 8480308.183),111706635,POINT (-75.24989 39.99370),POINT (-5707635.357 8481164.408)
645088,635.222698,3073,15165,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),3425873704,POINT (-75.26671 39.96489),POINT (-5712894.247 8481748.976)
648979,716.791621,3073,15166,13779,POINT (-75.26717 39.96867),POINT (-5712310.261 8481998.913),5544386939,POINT (-75.26643 39.96443),POINT (-5712957.523 8481690.948)


In [16]:
start_time = time.time()

# Reinstate geographic elements
gp_entry = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
gp_entry['Grid_m_centroid'] = gpd.GeoSeries(gp_entry['Grid_m_centroid'], crs = 3043)
gp_entry['Parkroad_coords_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_coords_centroid'], crs = 4326)
gp_entry['Parkroad_m_centroid'] = gpd.GeoSeries(gp_entry['Parkroad_m_centroid'], crs = 3043)

# Get index for merging later
roads = road_nodes.reset_index()

# Get the nearest entrance point for the grid centroids
mat2 = np.repeat(None,len(gp_entry))
for i in range(len(gp_entry)):
    nearest = int(roads['geometry'].sindex.nearest(gp_entry['Grid_coords_centroid'].iloc[i])[1])
    mat2[i] = roads['osmid'].iloc[nearest]
    if i % 100000 == 0: print(round(i/len(gp_entry)*100,1),'% done', round((time.time() - start_time) / 60,2),' mns')
print(len(mat2[mat2 != None]))

# Add all the information and the distance from the nearest point to the grid centroid in a geodataframe
pd.DataFrame(mat2)
gp_entry['grid_osm'] = mat2
gp_entry = pd.merge(gp_entry, road_nodes['geometry'], left_on = 'grid_osm', right_index = True)
gp_entry = gp_entry.loc[:, ~gp_entry.columns.isin(['grid_lon', 'grid_lat', 'park_lon', 'park_lat'])]
gp_entry['grid_entry_dist'] = round(gp_entry['Grid_m_centroid'].distance(gp_entry['geometry'].to_crs(3043)),3)
gp_entry = gp_entry.reset_index()
gp_entry

0.0 % done 0.0  mns
43.9 % done 0.15  mns
87.7 % done 0.31  mns
227969


,index,euclidean,Grid_No,Park_No,Grid_Id,Grid_coords_centroid,Grid_m_centroid,Parkroad_osmid,Parkroad_coords_centroid,Parkroad_m_centroid,grid_osm,geometry,grid_entry_dist
0,1151,710.210312,1151,0,6972,POINT (-75.24696 40.05156),POINT (-5698301.554 8483771.980),8627877668,POINT (-75.25214 40.05009),POINT (-5698738.706 8484331.709),4771110405,POINT (-75.24815 40.05287),268.020
1,5042,806.358546,1151,1,6972,POINT (-75.24696 40.05156),POINT (-5698301.554 8483771.980),5005312359,POINT (-75.25169 40.05472),POINT (-5697984.550 8484513.413),4771110405,POINT (-75.24815 40.05287),268.020
2,8933,698.181434,1151,2,6972,POINT (-75.24696 40.05156),POINT (-5698301.554 8483771.980),4244000096,POINT (-75.25031 40.05484),POINT (-5697910.887 8484350.631),4771110405,POINT (-75.24815 40.05287),268.020
3,12824,751.956564,1151,3,6972,POINT (-75.24696 40.05156),POINT (-5698301.554 8483771.980),5005312355,POINT (-75.25074 40.05499),POINT (-5697904.199 8484410.375),4771110405,POINT (-75.24815 40.05287),268.020
4,16715,750.384119,1151,4,6972,POINT (-75.24696 40.05156),POINT (-5698301.554 8483771.980),6651751695,POINT (-75.25030 40.05524),POINT (-5697847.121 8484369.113),4771110405,POINT (-75.24815 40.05287),268.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227964,378847,956.687216,1420,15097,7683,POINT (-75.24471 40.04293),POINT (-5699587.531 8483054.833),7385265866,POINT (-75.25214 40.04262),POINT (-5699928.888 8483948.548),5550942667,POINT (-75.24381 40.04170),235.269
227965,382738,947.467556,1420,15098,7683,POINT (-75.24471 40.04293),POINT (-5699587.531 8483054.833),7385317121,POINT (-75.25207 40.04268),POINT (-5699916.167 8483943.480),5550942667,POINT (-75.24381 40.04170),235.269
227966,386629,936.369342,1420,15099,7683,POINT (-75.24471 40.04293),POINT (-5699587.531 8483054.833),7385317120,POINT (-75.25199 40.04276),POINT (-5699900.629 8483937.306),5550942667,POINT (-75.24381 40.04170),235.269
227967,390520,983.501974,1420,15100,7683,POINT (-75.24471 40.04293),POINT (-5699587.531 8483054.833),7385265867,POINT (-75.25236 40.04302),POINT (-5699874.102 8483995.659),5550942667,POINT (-75.24381 40.04170),235.269


In [17]:
# Get only necessary road connections columns for network performance
road_conn = road_connections[['osmid','key','length','geometry']]
road_conn = road_conn.set_index('key')

In [18]:
#
gp_entry['Grid_m_centroid'].distance(gp_entry['Parkroad_m_centroid'].to_crs(3043))

0         710.210312
1         806.358546
2         698.181434
3         751.956564
4         750.384119
             ...    
227964    956.687216
227965    947.467556
227966    936.369342
227967    983.501974
227968    913.539094
Length: 227969, dtype: float64

In [20]:
import warnings
warnings.filterwarnings("ignore")

start_time = time.time()

parknode = list(gp_entry['Parkroad_osmid'])
gridnode = list(gp_entry['grid_osm'])

s_mat = list([])
s_mat1 = list([])
s_mat2 = list([])
s_mat3 = list([])
s_mat4 = list([])

for i in range(len(gp_entry)):
    # First try from population grid to park Dijkstra algorithm
    try:
        shortest = nx.shortest_path(graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
        s_mat.append(shortest)
        shortest_to = list(shortest[1:len(shortest)])
        shortest_to.append(0)
        s_mat1.append(shortest_to)
        s_mat2.append(list(np.repeat(i, len(shortest))))
        s_mat3.append(list(np.arange(0, len(shortest))))
        s_mat4.append('normal way')
        
    except:
        try:
            # Check the reverse
            shortest = nx.shortest_path(graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
            s_mat.append(shortest)
            shortest_to = list(shortest[1:len(shortest)])
            shortest_to.append(0)
            s_mat1.append(shortest_to)
            s_mat2.append(list(np.repeat(i, len(shortest))))
            s_mat3.append(list(np.arange(0, len(shortest))))
            s_mat4.append('reverse way')
        except:
            try:
                # Get the route to the nearest park node
                new_node = pd.DataFrame((abs(gp_entry['Parkroad_coords_centroid'].x[123381] - PtR['geometry_y'].x)**2
                                         + abs(gp_entry['Parkroad_coords_centroid'].y[123381] - PtR['geometry_y'].y)**2
                                        )**(1/2)).join(PtR['osmid']).sort_values(0).iloc[1,1]
                shortest = nx.shortest_path(graph, gridnode[i], new_node, 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('nearest way')
            except:
                # Print if none of the above three options gives any result
                print('index',i,'no route possible between osmid',gridnode[i],'and',parknode[i])
                pass
            
    if i % 10000 == 0: print(round(i/len(gp_entry)*100,2),'% done', round((time.time() - start_time) / 60,2),' mns')
print('100 % pathfinding done', round((time.time() - start_time) / 60,2),' mns')




0.0 % done 0.0  mns
4.39 % done 0.1  mns
8.77 % done 0.25  mns
13.16 % done 0.37  mns
17.55 % done 0.5  mns
21.93 % done 0.65  mns
26.32 % done 0.79  mns
30.71 % done 0.92  mns
35.09 % done 1.03  mns
39.48 % done 1.13  mns
43.87 % done 1.27  mns
48.25 % done 1.41  mns
52.64 % done 1.54  mns
57.03 % done 1.75  mns
61.41 % done 1.86  mns
65.8 % done 1.99  mns
70.18 % done 2.08  mns
74.57 % done 2.25  mns
78.96 % done 2.4  mns
83.34 % done 2.51  mns
87.73 % done 2.67  mns
92.12 % done 2.79  mns
96.5 % done 2.91  mns
100 % pathfinding done 3.0  mns


In [21]:
start_time = time.time()
# Unpack lists and add them in a dataframe
s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()

# Format df
routes.columns = ['from','to','route','step']
mat_key = list([])
for i in range(len(routes)):
    mat_key.append(str(int(s_mat_u[i])) + '-' + str(int(s_mat_u1[i])))
routes['key'] = mat_key
routes = routes.set_index('key')

# Add route information
routes = routes.join(road_conn, how = 'left')
routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
routes.sort_values(by = ['route','step'])
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes

time taken 1.3 mns


,from,to,route,step,osmid,length,geometry
key,,,,,,,
1015348767-349355149,1015348767,349355149,11470,0,87325393,23.887,"LINESTRING (-75.15005 39.94832, -75.14977 39.9..."
1015348767-349355149,1015348767,349355149,11845,0,87325393,23.887,"LINESTRING (-75.15005 39.94832, -75.14977 39.9..."
1015348767-349355149,1015348767,349355149,12772,0,87325393,23.887,"LINESTRING (-75.15005 39.94832, -75.14977 39.9..."
1015348767-349355149,1015348767,349355149,13408,0,87325393,23.887,"LINESTRING (-75.15005 39.94832, -75.14977 39.9..."
1015348767-349355149,1015348767,349355149,13890,0,87325393,23.887,"LINESTRING (-75.15005 39.94832, -75.14977 39.9..."
...,...,...,...,...,...,...,...
989146009-4696697785,989146009,4696697785,98521,3,344740382,2.457,"LINESTRING (-75.18702 39.95520, -75.18705 39.9..."
989146009-4696697785,989146009,4696697785,98525,3,344740382,2.457,"LINESTRING (-75.18702 39.95520, -75.18705 39.9..."
989146009-4696697785,989146009,4696697785,99117,3,344740382,2.457,"LINESTRING (-75.18702 39.95520, -75.18705 39.9..."


In [24]:
start_time = time.time()
# get single (dissolved) line per route, attach information.
routes2 = routes[['route','geometry']].dissolve('route')
routes2['way_calculated'] = s_mat4
routes2['route_cost'] = routes.groupby('route')['length'].sum()
routes2['num_steps'] = routes.groupby('route')['step'].max()
routes2.index = routes2.index.astype(int)
routes2 = pd.merge(routes2, gp_entry[['Grid_No','Park_No','grid_entry_dist','Parkroad_osmid','grid_osm']],
                   left_index = True, right_index = True)
routes2['total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
print('time taken', round((time.time() - start_time) / 60,2), 'mns')
routes2  
 

time taken 0.83 mns


,geometry,way_calculated,route_cost,num_steps,Grid_No,Park_No,grid_entry_dist,Parkroad_osmid,grid_osm,total_cost,gridpark_no
route,,,,,,,,,,,
0,"LINESTRING (-75.25214 40.05009, -75.25197 40.0...",normal way,459.237,1,1151,0,268.020,8627877668,4771110405,727.257,1151-0
1,"MULTILINESTRING ((-75.24815 40.05287, -75.2472...",normal way,720.876,5,1151,1,268.020,5005312359,4771110405,988.896,1151-1
2,"MULTILINESTRING ((-75.24815 40.05287, -75.2472...",normal way,584.483,3,1151,2,268.020,4244000096,4771110405,852.503,1151-2
3,"MULTILINESTRING ((-75.24815 40.05287, -75.2472...",normal way,624.890,4,1151,3,268.020,5005312355,4771110405,892.910,1151-3
4,"MULTILINESTRING ((-75.24815 40.05287, -75.2472...",normal way,628.731,4,1151,4,268.020,6651751695,4771110405,896.751,1151-4
...,...,...,...,...,...,...,...,...,...,...,...
227964,"MULTILINESTRING ((-75.24675 40.04394, -75.2469...",normal way,6947.083,17,1420,15097,235.269,7385265866,5550942667,7182.352,1420-15097
227965,"MULTILINESTRING ((-75.24675 40.04394, -75.2469...",normal way,6956.168,18,1420,15098,235.269,7385317121,5550942667,7191.437,1420-15098
227966,"MULTILINESTRING ((-75.24675 40.04394, -75.2469...",normal way,6967.261,19,1420,15099,235.269,7385317120,5550942667,7202.530,1420-15099


In [25]:
start_time = time.time()
# Get the entry point of the park that is closest to the grid centroid
gridpark_score = pd.DataFrame(routes2.groupby('gridpark_no')['total_cost'].min())
gridpark_score = pd.merge(gridpark_score, routes2.loc[:, ~routes2.columns.isin(['total_cost'])], 
                          left_index = True, right_on = 'gridpark_no')

# Check if the route to the closest entry point is within 1000m.
gridpark_score = gridpark_score[gridpark_score['total_cost'] <= 1000]

# The score is the inverse of the total cost (if a park is 100m (on route) away its score is 900 (1000 - 100))
gridpark_score['score'] = 1000 - gridpark_score['total_cost']

# Get the population weighted score
gridpark_score = pd.merge(gridpark_score, popgrid[['PoP2015_Number','geometry']],
                          left_on = 'Grid_No', right_index = True, how = 'outer')

gridpark_score['score'] = gridpark_score['score'].fillna(0)
gridpark_score['population_score'] = gridpark_score['score'] * gridpark_score['PoP2015_Number']

gridpark_score['Park_No'] = gridpark_score['Park_No'].fillna(-1)
gridpark_score['Park_No'] = gridpark_score['Park_No'].astype(int)

gridpark_score.index = gridpark_score.index.fillna(-1)
gridpark_score.index = gridpark_score.index.astype(int)

gridpark_score['num_steps'] = gridpark_score['num_steps'].fillna(-1)
gridpark_score['num_steps'] = gridpark_score['num_steps'].astype(int)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')
gridpark_score

time taken 0.01 mns


,total_cost,geometry_x,way_calculated,route_cost,num_steps,Grid_No,Park_No,grid_entry_dist,Parkroad_osmid,grid_osm,gridpark_no,score,PoP2015_Number,geometry_y,population_score
222202,792.024,"MULTILINESTRING ((-75.00833 40.12855, -75.0087...",normal way,744.809,6,10,13463,47.215,1.105504e+08,109892511,10-13463,207.976,127,"POLYGON ((-75.00778 40.13004, -75.00778 40.128...",26412.952
222203,696.249,"MULTILINESTRING ((-75.00833 40.12855, -75.0087...",normal way,649.034,5,10,13464,47.215,1.100560e+08,109892511,10-13464,303.751,127,"POLYGON ((-75.00778 40.13004, -75.00778 40.128...",38576.377
142999,95.495,"MULTILINESTRING ((-74.99950 40.05785, -74.9995...",normal way,74.730,2,1000,8149,20.765,4.785312e+09,7826688686,1000-8149,904.505,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",169142.435
143000,85.860,"LINESTRING (-74.99950 40.05785, -74.99952 40.0...",normal way,65.095,1,1000,8150,20.765,4.785312e+09,7826688686,1000-8150,914.140,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",170944.180
143001,200.578,"MULTILINESTRING ((-74.99944 40.05778, -74.9988...",normal way,179.813,4,1000,8151,20.765,7.827815e+09,7826688686,1000-8151,799.422,187,"POLYGON ((-74.99880 40.05932, -74.99880 40.057...",149491.914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
-1,NaN,None,NaN,NaN,-1,3877,-1,NaN,NaN,NaN,NaN,0.000,23,"POLYGON ((-75.24808 39.89000, -75.24808 39.888...",0.000
-1,NaN,None,NaN,NaN,-1,3878,-1,NaN,NaN,NaN,NaN,0.000,15,"POLYGON ((-75.24583 39.89000, -75.24583 39.888...",0.000
-1,NaN,None,NaN,NaN,-1,3880,-1,NaN,NaN,NaN,NaN,0.000,19,"POLYGON ((-75.24808 39.88827, -75.24808 39.886...",0.000
-1,NaN,None,NaN,NaN,-1,3881,-1,NaN,NaN,NaN,NaN,0.000,7,"POLYGON ((-75.24583 39.88827, -75.24583 39.886...",0.000


In [26]:
start_time = time.time()
# Group the park scores per grid.
grid_score = pd.DataFrame(gridpark_score.groupby('Grid_No')['score'].sum().sort_values(ascending=False))
grid_score['population_score (mln)'] = gridpark_score.groupby('Grid_No')['population_score'].sum()/1000000
grid_score['count'] = gridpark_score.groupby('Grid_No')['score'].count()

grid_score = grid_score.join(popgrid[['geometry','PoP2015_Number']])
grid_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Grid_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

# The merge contains grids with no access to parks within 1000m, these scores are set to 0
grid_score = gpd.GeoDataFrame(grid_score, geometry = 'geometry', crs = 4326)

print('time taken', round((time.time() - start_time) / 60,2), 'mns')
grid_score

time taken 1.66 mns


,score,population_score (mln),count,geometry,PoP2015_Number,lines
Grid_No,,,,,,
3411,218979.016,90.438334,385,"POLYGON ((-75.14926 39.94879, -75.14926 39.947...",413,"(LINESTRING (-75.1477891 39.944364300000004, -..."
3412,216109.298,82.337643,419,"POLYGON ((-75.14702 39.94879, -75.14702 39.947...",381,"(LINESTRING (-75.1477891 39.944364300000004, -..."
3193,213463.972,78.768206,477,"POLYGON ((-75.17397 39.96262, -75.17397 39.960...",369,"(LINESTRING (-75.1719176 39.9592884, -75.17204..."
3119,210578.690,169.515845,448,"POLYGON ((-75.17846 39.96780, -75.17846 39.966...",805,"(LINESTRING (-75.1792541 39.9662919, -75.17936..."
3172,196599.998,43.055400,507,"POLYGON ((-75.17621 39.96435, -75.17621 39.962...",219,"(LINESTRING (-75.1755483 39.95919299999999, -7..."
...,...,...,...,...,...,...
1248,0.000,0.000000,1,"POLYGON ((-75.23460 40.04897, -75.23460 40.047...",137,None
1249,0.000,0.000000,1,"POLYGON ((-75.23236 40.04897, -75.23236 40.047...",199,None
1278,0.000,0.000000,1,"POLYGON ((-75.08638 40.04897, -75.08638 40.047...",645,None


In [27]:
start_time = time.time()
# group the grid scores per park (which parks will be popular)
park_score = pd.DataFrame(gridpark_score.groupby('Park_No')['score'].sum().sort_values(ascending=False))
park_score['population_score (mln.)'] = gridpark_score.groupby('Park_No')['population_score'].sum()/1000000
park_score['count'] = gridpark_score.groupby('Park_No')['score'].count()

park_score['lines'] = gpd.GeoDataFrame(gridpark_score[['Park_No','geometry_x']], geometry = 'geometry_x', crs = 4326).dissolve('Park_No')

park_score = park_score.join(park_to_road['geometry'])

print('time taken', round((time.time() - start_time) / 60,2), 'mns')
park_score

time taken 0.31 mns


,score,population_score (mln.),count,lines,geometry
Park_No,,,,,
10394,9618.074,5.387857,24,"(LINESTRING (-75.2265561 39.95194810000001, -7...","POLYGON ((-75.22325 39.94787, -75.22327 39.947..."
10396,9609.044,5.436354,25,"(LINESTRING (-75.2265561 39.95194810000001, -7...","POLYGON ((-75.22325 39.94787, -75.22327 39.947..."
10395,9599.191,5.388218,24,"(LINESTRING (-75.2265561 39.95194810000001, -7...","POLYGON ((-75.22325 39.94787, -75.22327 39.947..."
10401,9462.317,5.383728,26,"(LINESTRING (-75.2265561 39.95194810000001, -7...","POLYGON ((-75.22325 39.94787, -75.22327 39.947..."
10391,9461.758,5.344295,25,"(LINESTRING (-75.2265561 39.95194810000001, -7...","POLYGON ((-75.22325 39.94787, -75.22327 39.947..."
...,...,...,...,...,...
5936,5.735,0.000338,1,"(LINESTRING (-75.2144944 39.982121600000006, -...","POLYGON ((-75.19009 39.94360, -75.19010 39.943..."
7862,4.518,0.000456,1,"(LINESTRING (-75.1989268 40.00006919999999, -7...","POLYGON ((-75.19009 39.94360, -75.19010 39.943..."
5943,3.961,0.000234,1,"(LINESTRING (-75.2144944 39.982121600000006, -...","POLYGON ((-75.19009 39.94360, -75.19010 39.943..."


In [28]:
park_lines = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'lines']], 
                              geometry = 'lines', crs = 4326)
park_lines.to_file(r'C:\Users\bartb\Downloads\park_entrance_routes.shp')

park_s = gpd.GeoDataFrame(park_score[['score', 'population_score (mln.)','count', 'geometry']], 
                          geometry = 'geometry', crs = 4326)
park_s.to_file(r'C:\Users\bartb\Downloads\park_score_entrance.shp')

In [29]:
grid_lines = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','lines']], 
                              geometry = 'lines', crs = 4326)
grid_lines.to_file(r'C:\Users\bartb\Downloads\grid_entrance_routes.shp')

grid_s = gpd.GeoDataFrame(grid_score[['score', 'population_score (mln)','count','PoP2015_Number','geometry']], 
                          geometry = 'geometry', crs = 4326)
grid_s.to_file(r'C:\Users\bartb\Downloads\grid_score_entrance.shp')

In [30]:
print(round((time.time() - start) / 60,2),'mns')

19.64 mns


In [ ]:
time.time()